<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# Tunable Parameters for the NISAR GCOV processor
#### Kernel: plant
The L2_L_GCOV standard product is produced using the NISAR GCOV processor driven by a standard set of configuration parameters. The **on-demand/algorithm development environment (ADE)** provides users with the capability to tweak these parameters and run them on an **on-demand NISAR Science Data System (SDS)** to generate custom L2_L_GOV products:

- *fullcovariance*: {optional} Compute cross-elements (True) or diagonals only (False). Default: False
- *output_type*: {optional} Choices: “None” (to turn off RTC) or “gamma0”; The *output_type* indicates the type of RTC that should be applied.
- *algorithm_type*: {optional} Choices: "area-projection" (default) and "David-Small"; The *algorithm_type* defines the algorithm used for computing the RTC area factor.
- *abs_rad_cal*: {optional} Absolute radiometric correction factor. Default 1
- *output_posting*: {required} Output posting in same units as output EPSG. Single value or list indicating the output posting for each frequency. Default "[20, 100]"; The *output_posting* is the distance between pixels of the output geogrid. *output_posting* and the *outputEPSG* spatial reference system parameter must have the same units. If the outputEPSG is not set, the output EPSG will be the same as the input DEM. The default value for *output_posting* is given by the list [20, 100], which defines 20m posting for the main band (frequency A) and 100m posting for the auxiliary band (frequency B). It is the responsibility of the user to ensure that *output_posting* values are consistent with *outputEPSG* (or DEMFile if applicable).

In this tutorial, we show you:
- how to submit a GCOV processor job to the **on-demand NISAR SDS** using an input L1_L_RSLC dataset to generate a standard L2_L_GCOV product using the default configuration parameters
- submit a second GCOV processor job to the **on-demand NISAR SDS** using the same input L1_L_RSLC dataset to generate a custom L2_L_GCOV product using tweaked parameters

<span style="color:red">**NOTE**: the L2_L_GCOV products generated in this notebook were produced by the R1.1.1 release (March 2021) of the GCOV processor (and its associated test data) which does not provide full-covariance matrix capabilities. Any visualizations will accordingly show no data.</span>

## Setup configuration for interaction with on-demand NISAR SDS
In this section, we setup the necessary python libraries and API endpoints needed to interact with the **on-demand NISAR SDS**. We print out the version of the installed **PLAnT** package.

The `otello` python library provides high-level access to operations on the on-demand SDS. In particular for this demo, it provides us with access to and information about the job types registered on the SDS along with the capability to submit jobs, check for job statuses, and to query for products generated by the job.

The `pele` python library provides high-level read-only access to the dataset catalog managed by the on-demand SDS. In particular for this demo, it provides us with the capability to list the dataset types contained in the catalog, query for individual dataset granules of a particular dataset type, and to return metadata pertaining to an individual dataset granule.

### What version of PLAnT is installed in the on-demand environment?

In [ ]:
import os

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    !mkdir -p ./notebook_output/05-Tunable_Parameters
    os.chdir('notebook_output/05-Tunable_Parameters')
    
import json
import time
import re
import shutil
from glob import glob
from pprint import pprint
from datetime import datetime
from pip._internal.commands.show import search_packages_info
import logging
import matplotlib
logging.getLogger('matplotlib').setLevel(logging.WARNING)

import otello
from pele_client.client import PeleRequests

import plant
%matplotlib inline

print(f"PLAnT version: {plant.__version__}")

### Configure for interaction with the on-demand NISAR SDS

In [ ]:
# set the mozart IP
mozart_ip = "100.64.122.181"
#mozart_ip = input("Enter IP address of your mozart instance then press <Enter>: ")
print(f"Using mozart IP address {mozart_ip}.")

# instantiate otello and get mozart object to interact with SDS
otello.client.initialize()
m = otello.mozart.Mozart()

# set pele URL
pele_url = f"https://{mozart_ip}/pele/api/v0.1"
print(f"Using pele URL {pele_url}.")

# instantiate PeleRequests object
pr = PeleRequests(pele_url, verify=False)

## Submitting an "on-demand standard product" job for the NISAR GCOV processor
### Next let's get more information on the NISAR GCOV processor:

In [ ]:
# get GCOV job type and initialize
job_type = "job-SCIFLO_GCOV:jdyoung-test"
gcov = m.get_job_types()[job_type]
gcov.initialize()

### Let's print the input parameters for the GCOV processor:

In [ ]:
print(gcov.describe())

### Query NISAR SDS for input RSLC dataset
The input dataset for the NISAR GCOV processor is the NISAR RSLC product. Here we query the on-demand NISAR SDS for such an input dataset:

In [ ]:
# get input RSLC dataset
r = pr.get(f"{pele_url}/pele/dataset/NISAR_L1_PR_RSLC_001_073_D_051_2800_HH_20080218T062000_20080218T062016_D00200_P_F_001")

rslc_dataset = r.json()["result"]

In [ ]:
print(r.json())

### Prepare input for the on-demand GCOV processor

In [ ]:
# set dataset params
gcov.set_input_dataset(rslc_dataset)

### Show the current configured parameter values

In [ ]:
# what are my currently set parameters?
pprint(gcov.get_input_params())

### Submit the GCOV job to run in the on-demand NISAR SDS

In [ ]:
# submit job
job1 = gcov.submit_job(tag="gcov-test1-default-params")

### Let's wait for the job to complete generation of the standard GCOV product

In [ ]:
# wait for job
job1.wait_for_completion()

### Get information about the generated standard GCOV product from the GCOV job
#### The generated standard GCOV product is stored in the cloud next to the on-demand NISAR SDS

In [ ]:
std_prod = job1.get_generated_products()
print(json.dumps(std_prod, indent=2, sort_keys=True))

### Download the generated standard GCOV product from the cloud into this notebook
#### Here we use the AWS CLI to download the generated dataset.

Make sure your AWS credentials are valid.

In [ ]:
std_prod_url = re.sub(r'^s3://.+?/(.+)$', r's3://\1', std_prod[0]["urls"][-1]) # get s3 url
std_local_dir = os.path.basename(std_prod_url)

if os.path.isdir(std_local_dir): shutil.rmtree(std_local_dir)
!aws s3 sync $std_prod_url $std_local_dir

### Use PLAnT to visualize the standard GCOV product
Here we list the contents of the generated dataset. The actual GCOV product is the HD5 file.

In [ ]:
!ls -l $std_local_dir

In [ ]:
# read in GCOV standard product
import h5py

std_gcov_file = glob(f"{std_local_dir}/*.h5")[0]
print(std_gcov_file)
gcov_h5_file = h5py.File(std_gcov_file, 'r')

std_hh_obj = f"NETCDF:\"{std_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/HHHH"
std_hv_obj = f"NETCDF:\"{std_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/HVHV"
std_vv_obj = f"NETCDF:\"{std_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/VVVV"
std_vh_obj = f"NETCDF:\"{std_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/VHVH"

# options
options = {}
# options['mute'] = True
# options['force'] = True

# display options
display_options = options.copy()

# display RGB composition of polarizations HH, HV, VV
plant.display(std_hh_obj, std_hv_obj, std_vv_obj, title="RGB comp (standard)", **display_options)

# show cropped view of RGB composition
plant.display(std_hh_obj, std_hv_obj, std_vv_obj, title="RGB comp cropped (standard)", 
              rows="1000:2000", cols="1000:2000", **display_options)

In [ ]:
# show cropped view of HH and  HV polarizations
plant.display(std_hh_obj, title="HH pol (standard)", rows="1000:2000", cols="1000:2000", **display_options)
plant.display(std_hv_obj, title="HV pol (standard)", rows="1000:2000", cols="1000:2000", **display_options)

# show profiles
plant.display(std_hh_obj, std_hv_obj, profile_x=True, title='X profile (standard)', **display_options)
plant.display(std_hh_obj, std_hv_obj, profile_y=True, title='Y profile (standard)', **display_options)

## Submitting a "custom on-demand" job for the NISAR GCOV processor with tunable parameters
Next let's submit a GCOV job for the same input product but this time **tweaking 3 of the 4 tunable parameters diverging from the standard product definition**. We will modify the job parameters so that:

- **fullcovariance**: "True" (to compute cross-elements)
- **algorithm_type**: "bilinear_distribution" (vs. area-projection)
- **output_posting**: 20 (vs. "[20, 100]")

For the output_type parameter, we will just accept the default value.

The `prompt_input_params()` method will prompt the user to modify or accept the default value for each of the tunable parameters.

After setting the parameters, we print them again to show what was set.

### Prepare input for the on-demand GCOV processor and modify the tunable parameters

In [ ]:
# set submitter params
gcov.prompt_input_params() # this will prompt user for values for each submitter

### Show the current configured parameter values

In [ ]:
# what are my currently set parameters?
pprint(gcov.get_input_params())

### Submit the GCOV job with tuned parameters to run in the on-demand NISAR SDS
Every on-demand job can be associated with a tag as a value-added feature for improved tracking, billing, and other uses. In the following job submission, we will tag the job with info on the modified parameters.

In [ ]:
# submit job
job2 = gcov.submit_job(tag="gcov-test1-algo_david-small-fullcov_true-posting_20")

### Let's wait for the job to complete generation of the custom GCOV product

In [ ]:
# wait for job
job2.wait_for_completion()

### Get information about the generated custom GCOV product from the GCOV job
#### The generated custom GCOV product is stored in the cloud next to the on-demand NISAR SDS

In [ ]:
cust_prod = job2.get_generated_products()
print(json.dumps(cust_prod, indent=2, sort_keys=True))

### Download the generated custom GCOV product from the cloud into this notebook

In [ ]:
cust_prod_url = re.sub(r'^s3://.+?/(.+)$', r's3://\1', cust_prod[0]["urls"][-1]) # get s3 url
cust_local_dir = os.path.basename(cust_prod_url)
if os.path.isdir(cust_local_dir): shutil.rmtree(cust_local_dir)
!aws s3 sync $cust_prod_url $cust_local_dir

## Use PLAnT to visualize differences between the standard and custom GCOV products

In [ ]:
!ls $cust_local_dir

### Here we show images of the standard vs. custom GCOV products:

In [ ]:
# read in GCOV product with plant
cust_gcov_file = glob(f"{cust_local_dir}/*.h5")[0]
cust_hh_obj = f"NETCDF:\"{cust_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/HHHH"
cust_hv_obj = f"NETCDF:\"{cust_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/HVHV"
cust_vv_obj = f"NETCDF:\"{cust_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/VVVV"
cust_vh_obj = f"NETCDF:\"{cust_gcov_file}\"://science/LSAR/GCOV/grids/frequencyA/VHVH"

# display RGB composition of polarizations HH, HV, VV: standard product vs. custom product
plant.display(std_hh_obj, std_hv_obj, std_vv_obj, title="RGB comp (standard)", **display_options)
plant.display(cust_hh_obj, cust_hv_obj, cust_vv_obj, title="RGB comp (tuned)", **display_options)

# show cropped view of RGB composition: standard product vs. custom product
plant.display(std_hh_obj, std_hv_obj, std_vv_obj, title="RGB comp cropped (standard)",
              rows="1000:2000", cols="1000:2000", **display_options)
plant.display(cust_hh_obj, cust_hv_obj, cust_vv_obj, title="RGB comp cropped (tuned)",
              rows="1000:2000", cols="1000:2000", **display_options)

### Plot histograms of standard vs. custom products in dB

In [ ]:
# show histogram diffs between standard product and custom
plant.display(std_hh_obj, cust_hh_obj, title="HHHH", hist=True, db=True, name="std,tuned", **display_options)
plant.display(std_hv_obj, cust_hv_obj, title="HVHV", hist=True, db=True, name="std,tuned", **display_options)
plant.display(std_vv_obj, cust_vv_obj, title="VVVV", hist=True, db=True, name="std,tuned", **display_options)
plant.display(std_vh_obj, cust_vh_obj, title="VHVH", hist=True, db=True, name="std,tuned", **display_options)

### Plot X and Y profiles of standard vs. custom products

In [ ]:
plant.display(std_hh_obj, std_hv_obj, profile_x=True, title='X profile (standard)', **display_options)
plant.display(cust_hh_obj, cust_hv_obj, profile_x=True, title='X profile (tuned)', **display_options)

In [ ]:
plant.display(std_hh_obj, std_hv_obj, profile_y=True, title='Y profile (standard)', **display_options)
plant.display(cust_hh_obj, cust_hv_obj, profile_y=True, title='Y profile (tuned)', **display_options)

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.7.1 and above</font>